In [3]:

# 基于迁移学习的两阶段方法

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import copy
import os
import pandas as pd
from sklearn.metrics import mean_squared_error


from dataloader import create_all_dataloaders, create_category_dataloaders, create_transfer_dataloaders,ChronosDataset
from calculate import calculate_metrics, print_metrics_table,calculate_uncertainty_metrics
# 设置随机种子以确保可重复性
torch.manual_seed(42)
np.random.seed(42)

# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

使用设备: cuda


In [4]:
def predict_with_uncertainty(model, inputs, category, domain_idx=None, mc_samples=100, device='cuda'):
    """
    使用MC Dropout进行不确定性估计 - 适配DANN模型
    """
    model = model.to(device)
    inputs = inputs.to(device)
    category = category.to(device)
    model.train()  # 开启dropout以进行随机采样
    
    predictions = []
    for _ in range(mc_samples):
        # 对于DANN模型，忽略domain_idx参数
        outputs = model(inputs, category)
        
        if isinstance(outputs, tuple):
            outputs = outputs[0]  # 如果模型返回元组，取第一个元素
        
        predictions.append(outputs.detach())
    
    # 将预测堆叠为形状[mc_samples, batch, buildings, forecasts]
    try:
        stacked_preds = torch.stack(predictions, dim=0)
        
        # 计算平均值和标准差
        mean_pred = torch.mean(stacked_preds, dim=0)
        std_pred = torch.std(stacked_preds, dim=0)
        
        # 计算95%置信区间
        lower_bound = mean_pred - 1.96 * std_pred
        upper_bound = mean_pred + 1.96 * std_pred
        
        # 确保边界在有效范围内
        lower_bound = torch.clamp(lower_bound, 0, 1)
        upper_bound = torch.clamp(upper_bound, 0, 1)
        
        # 转换为CPU NumPy数组
        return (mean_pred.cpu().numpy(),
                lower_bound.cpu().numpy(),
                upper_bound.cpu().numpy(),
                std_pred.cpu().numpy())
    
    except Exception as e:
        print(f"处理MC采样结果时出错: {str(e)}")
        # 如果堆叠或其他操作失败，使用第一个样本作为预测
        try:
            mean_pred = predictions[0].cpu().numpy()
            std_pred = np.ones_like(mean_pred) * 0.1
            lower_bound = np.maximum(mean_pred - 1.96 * std_pred, 0)
            upper_bound = np.minimum(mean_pred + 1.96 * std_pred, 1)
            return mean_pred, lower_bound, upper_bound, std_pred
        except:
            # 最后的后备方案：返回零数组
            shape = list(inputs.shape)
            if len(shape) >= 3:
                shape[-1] = 1
            zeros = np.zeros(shape)
            return zeros, zeros, zeros, zeros
def calculate_transfer_metrics(source_features, target_features, source_outputs=None, target_outputs=None, baseline_predictions=None, targets=None):
    """
    计算迁移学习的综合评估指标
    
    Args:
        source_features: 源域特征 [batch_size, feature_dim]
        target_features: 目标域特征 [batch_size, feature_dim]
        source_outputs: 源域模型输出（可选）
        target_outputs: 目标域模型输出（可选）
        baseline_predictions: 基线模型预测（可选）
        targets: 真实目标值（可选，用于计算负迁移）
    """
    import traceback  # 导入traceback以便打印详细错误信息
    
    # 处理3D特征 - 将3D特征转换为2D
    if len(source_features.shape) == 3:  # [batch, seq_len, feature_dim]
        logging.info(f"检测到3D特征，进行展平: source_features.shape={source_features.shape}")
        # 方法2：平均所有时间步
        source_features = source_features.mean(dim=1)  # [batch, feature_dim]
        target_features = target_features.mean(dim=1)  # [batch, feature_dim]
        
        logging.info(f"展平后: source_features.shape={source_features.shape}, target_features.shape={target_features.shape}")
    
    # 确保特征维度匹配
    if source_features.size(1) != target_features.size(1):
        logging.warning(f"特征维度不匹配: source_features.shape={source_features.shape}, target_features.shape={target_features.shape}")
        return {
            'a_distance': 'N/A',
            'feature_alignment': 'N/A',
            'mmd': 'N/A',
            'sample_efficiency': None,
            'CC': 'N/A'  # 添加CC字段
        }
    
    # 检查特征数量是否过大，可能导致内存问题
    max_samples = 5000
    if source_features.size(0) > max_samples or target_features.size(0) > max_samples:
        logging.warning(f"特征数量过大，进行采样: source_features.shape={source_features.shape}, target_features.shape={target_features.shape}")
        
        if source_features.size(0) > max_samples:
            indices = torch.randperm(source_features.size(0))[:max_samples]
            source_features = source_features[indices]
        
        if target_features.size(0) > max_samples:
            indices = torch.randperm(target_features.size(0))[:max_samples]
            target_features = target_features[indices]
        
        logging.info(f"采样后: source_features.shape={source_features.shape}, target_features.shape={target_features.shape}")
    
    def calculate_mmd(x, y):
        """计算最大平均差异(MMD)"""
        try:
            x_kernel = torch.mm(x, x.t())
            y_kernel = torch.mm(y, y.t())
            xy_kernel = torch.mm(x, y.t())
            return x_kernel.mean() + y_kernel.mean() - 2 * xy_kernel.mean()
        except Exception as e:
            logging.error(f"计算MMD时出错: {str(e)}")
            return float('nan')
    
    def calculate_a_distance(source_features, target_features):
        """计算A-distance"""
        try:
            domain_classifier = nn.Sequential(
                nn.Linear(source_features.size(1), 50),
                nn.ReLU(),
                nn.Linear(50, 1)
            ).to(source_features.device)
            
            source_domain_labels = torch.ones(source_features.size(0), 1).to(source_features.device)
            target_domain_labels = torch.zeros(target_features.size(0), 1).to(source_features.device)
            
            features = torch.cat([source_features, target_features], dim=0)
            labels = torch.cat([source_domain_labels, target_domain_labels], dim=0)
            
            optimizer = torch.optim.Adam(domain_classifier.parameters())
            criterion = nn.BCEWithLogitsLoss()
            
            for _ in range(100):
                optimizer.zero_grad()
                preds = domain_classifier(features)
                loss = criterion(preds, labels)
                loss.backward()
                optimizer.step()
            
            with torch.no_grad():
                # 确保使用浮点数计算均值
                preds = torch.sigmoid(domain_classifier(features))
                predicted_labels = (preds > 0.5).float()
                error = (predicted_labels != labels).float().mean()
            
            return 2 * (1 - 2 * error)
        except Exception as e:
            logging.error(f"计算A-distance时出错: {str(e)}")
            traceback.print_exc()  # 打印详细错误信息
            return float('nan')
    
    def calculate_feature_alignment(source_features, target_features):
        """计算特征对齐质量"""
        try:
            source_norm = F.normalize(source_features, p=2, dim=1)
            target_norm = F.normalize(target_features, p=2, dim=1)
            similarity = torch.mm(source_norm, target_norm.t())
            return similarity.mean()
        except Exception as e:
            logging.error(f"计算特征对齐质量时出错: {str(e)}")
            return float('nan')
    
    # 添加计算相关系数(CC)的函数
    def calculate_correlation_coefficient(predictions, targets):
        """计算相关系数(CC)"""
        try:
            # 确保输入是numpy数组
            if isinstance(predictions, torch.Tensor):
                predictions = predictions.detach().cpu().numpy()
            if isinstance(targets, torch.Tensor):
                targets = targets.detach().cpu().numpy()
            
            # 展平数组
            predictions = np.array(predictions).flatten()
            targets = np.array(targets).flatten()
            
            # 确保长度相同
            min_length = min(len(predictions), len(targets))
            predictions = predictions[:min_length]
            targets = targets[:min_length]
            
            # 计算相关系数
            cc = np.corrcoef(predictions, targets)[0, 1]
            return cc
        except Exception as e:
            logging.error(f"计算相关系数时出错: {str(e)}")
            traceback.print_exc()
            return float('nan')

    try:
        metrics = {
            'a_distance': calculate_a_distance(source_features, target_features),
            'feature_alignment': calculate_feature_alignment(source_features, target_features),
            'mmd': calculate_mmd(source_features, target_features)
        }
        
        # 如果提供了目标域输出和真实目标值，计算相关系数
        if target_outputs is not None and targets is not None:
            metrics['CC'] = calculate_correlation_coefficient(target_outputs, targets)
        else:
            metrics['CC'] = float('nan')  # 如果没有提供必要数据，设置为NaN
            
    except Exception as e:
        logging.error(f"计算迁移学习指标时出错: {str(e)}")
        traceback.print_exc()  # 打印详细错误信息
        return {
            'a_distance': 'N/A',
            'feature_alignment': 'N/A',
            'mmd': 'N/A',
            'CC': 'N/A'  # 添加CC字段
        }
    
    # 如果提供了输出、基线预测和目标值，检测负迁移
    if target_outputs is not None and baseline_predictions is not None and targets is not None:
        try:
            # 确保所有输入都是张量并且形状匹配
            # 首先转换为numpy数组以便统一处理
            if isinstance(target_outputs, torch.Tensor):
                target_outputs = target_outputs.detach().cpu().numpy()
            if isinstance(baseline_predictions, torch.Tensor):
                baseline_predictions = baseline_predictions.detach().cpu().numpy()
            if isinstance(targets, torch.Tensor):
                targets = targets.detach().cpu().numpy()
            
            # 确保都是一维数组
            target_outputs = np.array(target_outputs).flatten()
            baseline_predictions = np.array(baseline_predictions).flatten()
            targets = np.array(targets).flatten()
            
            # 确保所有数组长度相同
            min_length = min(len(target_outputs), len(baseline_predictions), len(targets))
            target_outputs = target_outputs[:min_length]
            baseline_predictions = baseline_predictions[:min_length]
            targets = targets[:min_length]
            
            # 计算MSE损失
            target_loss = np.mean((target_outputs - targets) ** 2)
            baseline_loss = np.mean((baseline_predictions - targets) ** 2)
            
            # 打印调试信息
            logging.debug(f"target_loss: {target_loss}, type: {type(target_loss)}")
            logging.debug(f"baseline_loss: {baseline_loss}, type: {type(baseline_loss)}")
            
        except Exception as e:
            logging.error(f"计算负迁移指标时出错: {str(e)}")
            traceback.print_exc()  # 打印详细错误信息
    
    return metrics
def evaluate_model(model, test_loader, model_name="Model", baseline_model=None, device='cuda', domain_idx=None):
    """
    评估模型在测试集上的性能 - 适配DANN模型
    """
    model.eval()
    all_predictions = []
    all_targets = []
    all_lower_bounds = []
    all_upper_bounds = []
    all_uncertainties = []
    # 用于迁移学习评估的特征收集
    source_features = []
    target_features = []
    source_outputs = []
    target_outputs = []
    
    # 计算标准预测和不确定性估计
    with torch.no_grad():
        for inputs, targets, _, category_onehot in test_loader:
            inputs = inputs.float().to(device)
            targets = targets.float().to(device)
            category_onehot = category_onehot.float().to(device)
            
            try:
                # 获取特征表示（用于迁移学习评估）
                if isinstance(model, DANN):
                    try:
                        # 对于DANN模型，直接获取特征
                        features = model.get_features(inputs)
                        # 由于DANN没有显式的域区分，我们可以将相同的特征用于源域和目标域
                        source_features.append(features)
                        target_features.append(features)
                    except Exception as e:
                        print(f"提取DANN特征时出错: {str(e)}")
                
                # 使用MC Dropout估计不确定性
                mean_pred, lower_bound, upper_bound, uncertainty = predict_with_uncertainty(
                    model=model, 
                    inputs=inputs, 
                    category=category_onehot,
                    device=device, 
                    mc_samples=50
                )
                
                # 收集预测结果
                if isinstance(mean_pred, torch.Tensor):
                    mean_pred = mean_pred.cpu().numpy()
                
                # 尝试收集源域和目标域的输出
                try:
                    if isinstance(model, DANN):
                        # 对于DANN模型，我们可以将相同的预测用于源域和目标域
                        source_outputs.append(mean_pred)
                        target_outputs.append(mean_pred)
                except Exception as e:
                    print(f"收集DANN输出时出错: {str(e)}")
                
                # 收集预测、目标和不确定性信息
                all_predictions.append(mean_pred)
                all_targets.append(targets.cpu().numpy())
                all_lower_bounds.append(lower_bound)
                all_upper_bounds.append(upper_bound)
                all_uncertainties.append(uncertainty)
            except Exception as e:
                print(f"批次处理过程中发生错误: {str(e)}")
                print(f"跳过此批次")
                continue
    
    
    # 检查是否有有效预测
    if not all_predictions:
        print("警告: 没有收集到任何有效预测，无法评估模型")
        return {"Model": model_name, "Error": "无有效预测"}
    
    # 合并所有批次的预测和目标
    try:
        all_predictions = np.concatenate(all_predictions, axis=0)
        all_targets = np.concatenate(all_targets, axis=0)
    except Exception as e:
        print(f"合并预测和目标时出错: {str(e)}")
        return {"Model": model_name, "Error": f"合并数据失败: {str(e)}"}
    
    # 检查不确定性相关的列表是否为空或合并是否成功
    if not all_lower_bounds or not all_upper_bounds or not all_uncertainties:
        print("警告: 不确定性估计列表为空，使用默认值")
        # 创建默认的不确定性估计
        all_lower_bounds = all_predictions * 0.9  # 简单地使用预测值的90%作为下界
        all_upper_bounds = all_predictions * 1.1  # 预测值的110%作为上界
        all_uncertainties = np.ones_like(all_predictions) * 0.1  # 默认不确定性为0.1
    else:
        # 合并所有批次的不确定性估计
        try:
            all_lower_bounds = np.concatenate(all_lower_bounds, axis=0)
            all_upper_bounds = np.concatenate(all_upper_bounds, axis=0)
            all_uncertainties = np.concatenate(all_uncertainties, axis=0)
        except Exception as e:
            print(f"合并不确定性估计时出错: {str(e)}")
            # 如果合并失败，使用默认值
            all_lower_bounds = all_predictions * 0.9
            all_upper_bounds = all_predictions * 1.1
            all_uncertainties = np.ones_like(all_predictions) * 0.1
    
    # 展平数组以便计算指标
    # 确保所有数组具有相同的形状
    all_predictions_flat = all_predictions.reshape(-1)
    all_targets_flat = all_targets.reshape(-1)
    
    # 确保不确定性相关的数组与预测和目标具有相同形状
    # 如果形状不同，可能需要广播或重采样
    if all_lower_bounds.size != all_predictions_flat.size:
        print(f"警告: 下界形状 {all_lower_bounds.shape} 与预测形状 {all_predictions.shape} 不一致")
        # 重塑或截断
        all_lower_bounds = all_lower_bounds.reshape(-1)[:all_predictions_flat.size]
        all_upper_bounds = all_upper_bounds.reshape(-1)[:all_predictions_flat.size]
        all_uncertainties = all_uncertainties.reshape(-1)[:all_predictions_flat.size]
    else:
        all_lower_bounds = all_lower_bounds.reshape(-1)
        all_upper_bounds = all_upper_bounds.reshape(-1)
        all_uncertainties = all_uncertainties.reshape(-1)
    
    # 打印形状信息用于调试
    print(f"形状信息:")
    print(f"预测: {all_predictions_flat.shape}")
    print(f"目标: {all_targets_flat.shape}")
    print(f"下界: {all_lower_bounds.shape}")
    print(f"上界: {all_upper_bounds.shape}")
    print(f"不确定性: {all_uncertainties.shape}")
    
    # 如果提供了基线模型，获取基线预测
    baseline_predictions = None
    if baseline_model is not None:
        baseline_model.eval()
        all_baseline_predictions = []
        
        with torch.no_grad():
            for inputs, _, _, category_onehot in test_loader:
                try:
                    inputs = inputs.float().to(device)
                    category_onehot = category_onehot.float().to(device)
                    
                    # 根据基线模型类型获取预测
                    if hasattr(baseline_model, 'predict'):
                        # 如果基线模型有predict方法
                        baseline_pred = baseline_model.predict(inputs)
                    else:
                        # 尝试直接获取预测值
                        baseline_pred = baseline_model(inputs, category_onehot)
                        # 处理返回元组的情况
                        if isinstance(baseline_pred, tuple):
                            baseline_pred = baseline_pred[0]
                    
                    # 转换为NumPy数组
                    if isinstance(baseline_pred, torch.Tensor):
                        baseline_pred = baseline_pred.cpu().numpy()
                    
                    all_baseline_predictions.append(baseline_pred)
                except Exception as e:
                    print(f"获取基线预测时出错: {str(e)}")
                    continue
        
        # 合并所有批次的基线预测
        if all_baseline_predictions:
            try:
                baseline_predictions = np.concatenate(all_baseline_predictions, axis=0).reshape(-1)
                
                # 确保基线预测具有相同的大小
                if baseline_predictions.size != all_predictions_flat.size:
                    print(f"警告: 基线预测形状 {baseline_predictions.shape} 与预测形状 {all_predictions_flat.shape} 不一致")
                    baseline_predictions = baseline_predictions[:all_predictions_flat.size]
            except Exception as e:
                print(f"处理基线预测时出错: {str(e)}")
                baseline_predictions = None
    
    # 改进的MAPE计算函数
    def calculate_improved_mape(y_true, y_pred, epsilon=0.01):
        """
        计算改进的MAPE，忽略接近零的值
        
        参数:
        - y_true: 真实值数组
        - y_pred: 预测值数组
        - epsilon: 阈值，避免除以接近零的值，对于0-1区间的数据，0.01是合理的
        
        返回:
        - MAPE值(百分比)
        """
        mask = np.abs(y_true) > epsilon
        if not np.any(mask):
            return float('nan')  # 如果没有有效值，返回NaN
        
        return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    
    # 计算评估指标，包含基线预测和置信区间
    try:
        metrics = calculate_metrics(
            predictions=all_predictions_flat, 
            real_values=all_targets_flat, 
            model_name=model_name, 
            baseline_predictions=baseline_predictions,
            lower_bounds=all_lower_bounds,
            upper_bounds=all_upper_bounds,
            confidence=0.95
        )
        
        # 添加平均不确定性
        metrics['avg_uncertainty'] = np.mean(all_uncertainties)
        
        # 使用改进的MAPE计算
        try:
            metrics['MAPE'] = calculate_improved_mape(all_targets_flat, all_predictions_flat, epsilon=0.01)
        except Exception as e:
            print(f"计算改进MAPE时出错: {str(e)}")
            # 保留原始MAPE或设置为NaN
            if 'MAPE' not in metrics:
                metrics['MAPE'] = float('nan')
    
        # 计算迁移学习指标 - 仅当所有必要的列表都非空时
        if (source_features and target_features and 
            source_outputs and target_outputs):
            try:
                # 处理特征数据
                source_features = torch.cat(source_features, dim=0).cpu()
                target_features = torch.cat(target_features, dim=0).cpu()
                source_outputs = np.concatenate(source_outputs, axis=0)
                target_outputs = np.concatenate(target_outputs, axis=0)
    
                # 计算迁移学习相关指标
                transfer_metrics = calculate_transfer_metrics(
                    source_features=source_features,
                    target_features=target_features,
                    source_outputs=source_outputs,
                    target_outputs=target_outputs,
                    baseline_predictions=baseline_predictions,
                    targets=all_targets_flat
                )
                # 更新指标字典
                metrics.update(transfer_metrics)
            except Exception as e:
                print(f"计算迁移学习指标时出错: {str(e)}")
                # 不要让这个错误影响整个评估流程
        else:
            print("警告: 迁移学习特征或输出列表为空，跳过迁移学习指标计算")
    
    except Exception as e:
        print(f"计算指标时出错: {str(e)}")
        # 创建一个基本的指标集作为回退
        from sklearn.metrics import mean_squared_error, r2_score
        rmse = np.sqrt(mean_squared_error(all_targets_flat, all_predictions_flat))
        r2 = r2_score(all_targets_flat, all_predictions_flat)
        
        metrics = {
            'Model': model_name,
            'RMSD': rmse,
            'R2': r2,
            'avg_uncertainty': np.mean(all_uncertainties)
        }
        
        # 尝试计算改进的MAPE
        try:
            metrics['MAPE'] = calculate_improved_mape(all_targets_flat, all_predictions_flat, epsilon=0.01)
        except Exception as e:
            print(f"计算改进MAPE时出错: {str(e)}")
            metrics['MAPE'] = float('nan')
    
    # 打印详细的评估报告
    print(f"\n{model_name} 评估报告:")
    print(f"RMSD: {metrics.get('RMSD', 'N/A')}")
    print(f"MAPE: {metrics.get('MAPE', 'N/A')}%")
    print(f"R²: {metrics.get('R2', 'N/A')}")
    print(f"KL散度: {metrics.get('KL_divergence', 'N/A')}")
    
    # 打印迁移学习指标 - 安全处理格式化
    if any(key in metrics for key in ['a_distance', 'feature_alignment', 'mmd']):
        print("\n迁移学习评估:")
        
        # 安全处理a_distance
        a_distance = metrics.get('a_distance', 'N/A')
        if isinstance(a_distance, (int, float)) and not (isinstance(a_distance, float) and np.isnan(a_distance)):
            print(f"域间距离 (A-distance): {a_distance:.4f}")
        else:
            print(f"域间距离 (A-distance): {a_distance}")
        
        # 安全处理feature_alignment
        feature_alignment = metrics.get('feature_alignment', 'N/A')
        if isinstance(feature_alignment, (int, float)) and not (isinstance(feature_alignment, float) and np.isnan(feature_alignment)):
            print(f"特征对齐质量: {feature_alignment:.4f}")
        else:
            print(f"特征对齐质量: {feature_alignment}")
        
        # 安全处理mmd
        mmd = metrics.get('mmd', 'N/A')
        if isinstance(mmd, (int, float)) and not (isinstance(mmd, float) and np.isnan(mmd)):
            print(f"MMD: {mmd:.4f}")
        else:
            print(f"MMD: {mmd}")
        
  
    # 打印不确定性评估指标 - 安全处理
    if 'PICP' in metrics:
        print(f"\n不确定性评估:")
        
        picp = metrics.get('PICP', 'N/A')
        if isinstance(picp, (int, float)) and not (isinstance(picp, float) and np.isnan(picp)):
            print(f"预测区间覆盖率(PICP): {picp:.2f}% (目标95%)")
        else:
            print(f"预测区间覆盖率(PICP): {picp}% (目标95%)")
            
        nmpiw = metrics.get('NMPIW', 'N/A')
        if isinstance(nmpiw, (int, float)) and not (isinstance(nmpiw, float) and np.isnan(nmpiw)):
            print(f"平均预测区间宽度(NMPIW): {nmpiw:.4f}")
        else:
            print(f"平均预测区间宽度(NMPIW): {nmpiw}")
            
        calibration_error = metrics.get('calibration_error', 'N/A')
        if isinstance(calibration_error, (int, float)) and not (isinstance(calibration_error, float) and np.isnan(calibration_error)):
            print(f"校准误差: {calibration_error:.2f}%")
        else:
            print(f"校准误差: {calibration_error}")
            

            
        avg_uncertainty = metrics.get('avg_uncertainty', 'N/A')
        if isinstance(avg_uncertainty, (int, float)) and not (isinstance(avg_uncertainty, float) and np.isnan(avg_uncertainty)):
            print(f"平均不确定性(标准差): {avg_uncertainty:.4f}")
        else:
            print(f"平均不确定性(标准差): {avg_uncertainty}")
    
    if 'PIR' in metrics and metrics['PIR'] is not None:
        pir = metrics.get('PIR', 'N/A')
        if isinstance(pir, (int, float)) and not (isinstance(pir, float) and np.isnan(pir)):
            print(f"\n相比基线的改进率:")
            print(f"RMSD改进率(PIR): {pir:.2f}%")
        else:
            print(f"\n相比基线的改进率:")
            print(f"RMSD改进率(PIR): {pir}")

    
    # 尝试可视化一个样本的预测与置信区间
    # 尝试可视化一个样本的预测与置信区间
    try:
        sample_idx = 0  # 选择第一个样本进行可视化
        
        # 检查数据维度并适应相应的可视化方法
        if len(all_predictions.shape) == 3 and all_predictions.shape[0] > 0 and all_predictions.shape[1] > 0 and all_predictions.shape[2] > 0:
            # 3D数据: [batch, building, forecast]
            max_horizon = min(10, all_predictions.shape[2])  # 限制显示的预测长度
            
            plt.figure(figsize=(10, 6))
            try:
                plt.fill_between(
                    range(max_horizon),
                    all_lower_bounds.reshape(all_predictions.shape)[sample_idx, 0, :max_horizon],
                    all_upper_bounds.reshape(all_predictions.shape)[sample_idx, 0, :max_horizon],
                    alpha=0.3, color='blue', label='95% Confidence Interval'
                )
                plt.plot(range(max_horizon), all_predictions[sample_idx, 0, :max_horizon], 'b-o', label='Predictions')
                plt.plot(range(max_horizon), all_targets[sample_idx, 0, :max_horizon], 'r-x', label='Ground Truth')
            except Exception as e:
                print(f"3D data visualization error: {str(e)}")
                # Try simplified visualization
                plt.plot(range(max_horizon), all_predictions[sample_idx, 0, :max_horizon], 'b-o', label='Predictions')
                plt.plot(range(max_horizon), all_targets[sample_idx, 0, :max_horizon], 'r-x', label='Ground Truth')
        
        # 2D数据的情况
        elif len(all_predictions.shape) == 2 and all_predictions.shape[0] > 0 and all_predictions.shape[1] > 0:
            max_horizon = min(10, all_predictions.shape[1])
            
            plt.figure(figsize=(10, 6))
            try:
                plt.fill_between(
                    range(max_horizon),
                    all_lower_bounds.reshape(all_predictions.shape)[sample_idx, :max_horizon],
                    all_upper_bounds.reshape(all_predictions.shape)[sample_idx, :max_horizon],
                    alpha=0.3, color='blue', label='95% Confidence Interval'
                )
                plt.plot(range(max_horizon), all_predictions[sample_idx, :max_horizon], 'b-o', label='Predictions')
                plt.plot(range(max_horizon), all_targets[sample_idx, :max_horizon], 'r-x', label='Ground Truth')
            except Exception as e:
                print(f"2D data visualization error: {str(e)}")
                # Try simplified visualization
                plt.plot(range(max_horizon), all_predictions[sample_idx, :max_horizon], 'b-o', label='Predictions')
                plt.plot(range(max_horizon), all_targets[sample_idx, :max_horizon], 'r-x', label='Ground Truth')
        
        # 1D数据的情况
        elif len(all_predictions.shape) == 1 and all_predictions.shape[0] > 0:
            max_points = min(10, all_predictions.shape[0])
            
            plt.figure(figsize=(10, 6))
            try:
                plt.fill_between(
                    range(max_points),
                    all_lower_bounds[:max_points],
                    all_upper_bounds[:max_points],
                    alpha=0.3, color='blue', label='95% Confidence Interval'
                )
                plt.plot(range(max_points), all_predictions[:max_points], 'b-o', label='Predictions')
                plt.plot(range(max_points), all_targets[:max_points], 'r-x', label='Ground Truth')
            except Exception as e:
                print(f"1D data visualization error: {str(e)}")
                # Try the simplest visualization
                plt.plot(range(max_points), all_predictions[:max_points], 'b-o', label='Predictions')
                plt.plot(range(max_points), all_targets[:max_points], 'r-x', label='Ground Truth')
        
        else:
            print(f"Cannot visualize predictions: incompatible shape {all_predictions.shape}")
            return metrics
        
        plt.title(f'{model_name} Prediction Example\nPICP: {metrics.get("PICP", "N/A")}%')
        plt.xlabel('Prediction Time Step')
        plt.ylabel('Value')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Save the image, ensure the file name is valid
        safe_model_name = ''.join(c if c.isalnum() or c in ['-', '_'] else '_' for c in model_name)
        plt.savefig(f'prediction_sample_{safe_model_name}.png')
        plt.close()

    except Exception as e:
        print(f"创建可视化时出错: {str(e)}")
        print(f"数据形状: 预测={all_predictions.shape}, 目标={all_targets.shape}, 下界={all_lower_bounds.shape}, 上界={all_upper_bounds.shape}")
    
    return metrics

In [5]:

from bilstm import BaselineBiLSTM
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import logging
from tqdm.auto import tqdm
import warnings
import copy
from torch import optim
from torch.autograd import Function

# 配置日志：同时输出到文件和控制台，记录时间、等级和消息
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log'),  # 日志文件
        logging.StreamHandler()               # 控制台输出
    ]
)

# 忽略警告
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# ========================= DANN模型定义 =========================

# 梯度反转层 - DANN的核心组件
class GradientReversalFunction(Function):
    """
    梯度反转层：前向传播时不变，反向传播时梯度反转
    """
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)
    
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None

class GradientReversal(nn.Module):
    """
    梯度反转层的包装模块
    """
    def __init__(self, alpha=1.0):
        super(GradientReversal, self).__init__()
        self.alpha = alpha
        
    def forward(self, x):
        return GradientReversalFunction.apply(x, self.alpha)

class FeatureEncoder(nn.Module):
    """
    特征编码器：提取时间序列特征
    """
    def __init__(self, input_dim, hidden_dim, num_layers=2, dropout=0.2):
        super(FeatureEncoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # 双向LSTM编码器
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # 特征投影层
        self.feature_projection = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        """
        参数:
        x: 输入张量 [batch_size, num_buildings, seq_len, input_dim]
        
        返回:
        features: 编码后的特征 [batch_size, num_buildings, seq_len, hidden_dim]
        """
        batch_size, num_buildings, seq_len, _ = x.shape
        
        # 重塑为[batch*buildings, seq_len, input_dim]
        x_reshaped = x.reshape(batch_size * num_buildings, seq_len, -1)
        
        # LSTM编码
        lstm_out, _ = self.lstm(x_reshaped)  # [batch*buildings, seq_len, hidden_dim*2]
        
        # 特征投影
        projected_features = self.feature_projection(lstm_out)  # [batch*buildings, seq_len, hidden_dim]
        
        # 重塑回[batch, buildings, seq_len, hidden_dim]
        features = projected_features.reshape(batch_size, num_buildings, seq_len, -1)
        
        return features

class LabelPredictor(nn.Module):
    """
    标签预测器：预测目标值
    """
    def __init__(self, feature_dim, category_dim, forecast_horizon, dropout=0.2):
        super(LabelPredictor, self).__init__()
        self.feature_dim = feature_dim
        self.forecast_horizon = forecast_horizon
        
        # 类别嵌入层
        self.category_embedding = nn.Sequential(
            nn.Linear(category_dim, feature_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # 预测层
        self.predictor = nn.Sequential(
            nn.Linear(feature_dim + feature_dim // 2, feature_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(feature_dim, forecast_horizon)
        )
    
    def forward(self, features, category_onehot):
        """
        参数:
        features: 编码后的特征 [batch_size, num_buildings, seq_len, feature_dim]
        category_onehot: 类别one-hot编码 [batch_size, category_dim]
        
        返回:
        predictions: 预测值 [batch_size, num_buildings, forecast_horizon]
        """
        batch_size, num_buildings, seq_len, _ = features.shape
        
        # 只使用最后一个时间步的特征
        last_step_features = features[:, :, -1, :]  # [batch, buildings, feature_dim]
        
        # 类别嵌入
        category_embedded = self.category_embedding(category_onehot)  # [batch, feature_dim//2]
        
        # 扩展类别嵌入以匹配建筑物维度
        category_expanded = category_embedded.unsqueeze(1).expand(-1, num_buildings, -1)  # [batch, buildings, feature_dim//2]
        
        # 连接特征和类别
        combined = torch.cat([last_step_features, category_expanded], dim=2)  # [batch, buildings, feature_dim + feature_dim//2]
        
        # 预测
        predictions = self.predictor(combined)  # [batch, buildings, forecast_horizon]
        
        return predictions

class DomainClassifier(nn.Module):
    """
    域分类器：区分源域和目标域
    """
    def __init__(self, feature_dim, hidden_dim=64, dropout=0.2):
        super(DomainClassifier, self).__init__()
        self.feature_dim = feature_dim
        
        # 域分类器网络
        self.classifier = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1)
        )
    
    def forward(self, features):
        """
        参数:
        features: 编码后的特征 [batch_size, feature_dim]
        
        返回:
        domain_pred: 域预测 [batch_size, 1]
        """
        return self.classifier(features)

class DANN(nn.Module):
    """
    Domain-Adversarial Neural Network (DANN)
    """
    def __init__(self, input_dim, hidden_dim, category_dim, forecast_horizon, 
                 num_buildings=1, num_layers=2, dropout=0.2, alpha=1.0):
        super(DANN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.category_dim = category_dim
        self.forecast_horizon = forecast_horizon
        self.num_buildings = num_buildings
        self.alpha = alpha
        self.num_layers = num_layers
        self.dropout = dropout
        
        # 特征编码器
        self.feature_encoder = FeatureEncoder(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            dropout=dropout
        )
        
        # 标签预测器
        self.label_predictor = LabelPredictor(
            feature_dim=hidden_dim,
            category_dim=category_dim,
            forecast_horizon=forecast_horizon,
            dropout=dropout
        )
        
        # 梯度反转层
        self.grl = GradientReversal(alpha=alpha)
        
        # 域分类器
        self.domain_classifier = DomainClassifier(
            feature_dim=hidden_dim,
            hidden_dim=hidden_dim,
            dropout=dropout
        )
    
    def forward(self, x, category_onehot, domain_idx=None):
        """
        参数:
        x: 输入张量 [batch_size, num_buildings, seq_len, input_dim]
        category_onehot: 类别one-hot编码 [batch_size, category_dim]
        domain_idx: 域索引，用于兼容接口，实际不使用
        
        返回:
        predictions: 预测值 [batch_size, num_buildings, forecast_horizon]
        """
        # 特征提取
        features = self.feature_encoder(x)
        
        # 标签预测
        predictions = self.label_predictor(features, category_onehot)
        
        return predictions
    
    def get_features(self, x):
        """
        获取特征表示，用于域对抗训练
        
        参数:
        x: 输入张量 [batch_size, num_buildings, seq_len, input_dim]
        
        返回:
        features: 编码后的特征 [batch_size*num_buildings, hidden_dim]
        """
        batch_size, num_buildings, seq_len, _ = x.shape
        
        # 特征提取
        features = self.feature_encoder(x)  # [batch, buildings, seq_len, hidden_dim]
        
        # 只使用最后一个时间步的特征并展平
        last_step_features = features[:, :, -1, :]  # [batch, buildings, hidden_dim]
        flattened_features = last_step_features.reshape(batch_size * num_buildings, -1)
        
        return flattened_features
    
    def domain_classify(self, features):
        """
        域分类，用于域对抗训练
        
        参数:
        features: 编码后的特征 [batch_size, feature_dim]
        
        返回:
        domain_pred: 域预测 [batch_size, 1]
        """
        # 应用梯度反转
        reversed_features = self.grl(features)
        
        # 域分类
        domain_pred = self.domain_classifier(reversed_features)
        
        return domain_pred
    
    def set_alpha(self, alpha):
        """
        设置梯度反转层的alpha值
        
        参数:
        alpha: 梯度反转系数
        """
        self.alpha = alpha
        self.grl.alpha = alpha

# ========================= 损失函数和评估函数 =========================

# 定义Huber损失函数
class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super(HuberLoss, self).__init__()
        self.delta = delta
    
    def forward(self, predictions, targets):
        # 计算预测值与目标值之间的差异
        diff = predictions - targets
        abs_diff = torch.abs(diff)
        
        # 应用Huber损失公式
        loss = torch.where(
            abs_diff <= self.delta,
            0.5 * diff * diff,
            self.delta * (abs_diff - 0.5 * self.delta)
        )
        
        return torch.mean(loss)

# 计算损失的辅助函数
def calculate_loss(predictions, targets):
    """
    计算Huber损失
    
    参数:
    - predictions: 预测值
    - targets: 真实值
    
    返回:
    - loss: Huber损失值
    """
    loss_fn = HuberLoss(delta=1.0)
    return loss_fn(predictions, targets)

# 简化评估函数，适配DANN模型
def simple_evaluate_model(model, test_loader, model_name="Model", device='cuda', domain_idx=None):
    """
    训练中使用的简化评估函数，适配DANN模型
    
    参数:
    - model: 要评估的模型
    - test_loader: 测试数据加载器
    - model_name: 模型名称
    - device: 计算设备
    - domain_idx: 域索引(不使用，保留参数兼容性)
    
    返回:
    - 简化的评估指标字典
    """
    from sklearn.metrics import mean_squared_error, r2_score
    
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets, _, category_onehot in test_loader:
            inputs = inputs.float().to(device)
            targets = targets.float().to(device)
            category_onehot = category_onehot.float().to(device)
            
            # 使用DANN模型进行预测
            predictions = model(inputs, category_onehot)
            
            # 收集预测和目标值
            pred_values = predictions.cpu().numpy()
            target_values = targets.cpu().numpy()
            
            all_preds.append(pred_values)
            all_targets.append(target_values)
    
    # 合并所有批次的预测和目标
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    
    # 展平数组以便计算指标
    all_preds = all_preds.reshape(-1)
    all_targets = all_targets.reshape(-1)
    
    # 计算基本指标
    mse = mean_squared_error(all_targets, all_preds)
    rmsd = np.sqrt(mse)
    r2 = r2_score(all_targets, all_preds)
    
    # 计算MAPE（排除零值）
    mask = np.abs(all_targets) > 1e-6
    mape = np.mean(np.abs((all_targets[mask] - all_preds[mask]) / all_targets[mask])) * 100 if np.any(mask) else np.nan
    
    # 计算CC (相关系数)
    cc = np.corrcoef(all_preds, all_targets)[0, 1]
    
    return {
        "Model": model_name,
        "RMSD": rmsd,
        "R2": r2,
        "MAPE": mape,
        "CC": cc
    }



In [6]:
def train_dann_source_model(model, train_loader, val_loader, epochs, lr, weight_decay, 
                           model_name, save_dir='models', device='cuda', early_stopping_patience=5):
    """
    训练DANN源域模型
    """
    os.makedirs(save_dir, exist_ok=True)
    model = model.to(device)
    
    # 设置优化器和学习率调度器
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    
    # 初始化记录器
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_epoch = 0
    patience_counter = 0
    
    # 保存模型配置信息
    model_config = {
        'input_dim': model.input_dim,
        'hidden_dim': model.hidden_dim,
        'category_dim': model.category_dim,
        'forecast_horizon': model.forecast_horizon,
        'num_buildings': model.num_buildings,
        'num_layers': model.num_layers,
        'dropout': model.dropout,
        'model_type': 'DANN'
    }
    
    # 保存最佳模型的信息
    best_model_info = {
        'state_dict': None,
        'optimizer_state': None,
        'epoch': 0,
        'train_loss': float('inf'),
        'val_loss': float('inf'),
        'metrics': None
    }

    print(f"开始训练 {model_name}...")

    for epoch in range(epochs):
        # 训练阶段
        model.train()
        epoch_train_loss = 0
        train_steps = 0
        
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Training]')
        for batch in progress_bar:
            inputs, targets, category, category_onehot = [
                x.float().to(device) if torch.is_tensor(x) else x for x in batch
            ]
            
            optimizer.zero_grad()
            
            # 前向传播 - 只进行标签预测任务
            predictions = model(inputs, category_onehot)
            
            # 计算任务损失
            loss = calculate_loss(predictions, targets)  # 使用Huber损失函数
            
            # 反向传播
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_train_loss += loss.item()
            train_steps += 1
            progress_bar.set_postfix({'train_loss': f'{loss.item():.4f}'})
        
        avg_train_loss = epoch_train_loss / train_steps
        train_losses.append(avg_train_loss)
        
        # 验证阶段
        model.eval()
        epoch_val_loss = 0
        val_steps = 0
        
        with torch.no_grad():
            progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Validation]')
            for batch in progress_bar:
                inputs, targets, category, category_onehot = [
                    x.float().to(device) if torch.is_tensor(x) else x for x in batch
                ]
                
                # 前向传播
                predictions = model(inputs, category_onehot)
                
                # 计算损失
                loss = calculate_loss(predictions, targets)
                
                epoch_val_loss += loss.item()
                val_steps += 1
                progress_bar.set_postfix({'val_loss': f'{loss.item():.4f}'})
        
        avg_val_loss = epoch_val_loss / val_steps
        val_losses.append(avg_val_loss)
        
        # 学习率调整
        scheduler.step(avg_val_loss)
        
        # 计算当前模型的评估指标
        current_metrics = simple_evaluate_model(model, val_loader, f"{model_name}_epoch_{epoch+1}", device=device)
        
        # 检查是否是最佳模型
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_epoch = epoch
            patience_counter = 0
            
            # 更新最佳模型信息
            best_model_info = {
                'state_dict': copy.deepcopy(model.state_dict()),
                'optimizer_state': copy.deepcopy(optimizer.state_dict()),
                'epoch': epoch + 1,
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'metrics': current_metrics,
                'hyperparameters': {
                    'lr': lr,
                    'weight_decay': weight_decay,
                    'epochs': epochs,
                    'best_epoch': epoch + 1,
                    'model_config': model_config,
                    'model_type': 'DANN'
                }
            }
            
            # 保存最佳模型检查点
            checkpoint_path = os.path.join(save_dir, f'{model_name}_best.pth')
            torch.save(best_model_info, checkpoint_path)
            print(f"✅ 保存最佳模型 (epoch {epoch+1}), 验证损失: {avg_val_loss:.4f}")
        else:
            patience_counter += 1
        
        # 打印当前epoch的训练信息
        print(
            f"Epoch {epoch+1}/{epochs} - "
            f"Train Loss: {avg_train_loss:.4f}, "
            f"Val Loss: {avg_val_loss:.4f}, "
            f"RMSD: {current_metrics['RMSD']:.4f}, "
            f"R²: {current_metrics['R2']:.4f}, "
            f"Best Val Loss: {best_val_loss:.4f} (Epoch {best_epoch+1}), "
            f"LR: {optimizer.param_groups[0]['lr']:.6f}"
        )
        
        # 早停检查
        if patience_counter >= early_stopping_patience:
            print(f"Early stopping triggered after epoch {epoch+1}")
            break
    
    # 训练结束后，保存训练历史
    history = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'best_epoch': best_epoch + 1,
        'best_val_loss': best_val_loss
    }
    
    # 保存训练历史
    history_path = os.path.join(save_dir, f'{model_name}_training_history.json')
    with open(history_path, 'w') as f:
        # 将列表转换为可序列化格式
        serializable_history = {
            'train_losses': [float(loss) for loss in train_losses],
            'val_losses': [float(loss) for loss in val_losses],
            'best_epoch': best_epoch + 1,
            'best_val_loss': float(best_val_loss)
        }
        json.dump(serializable_history, f, indent=4)
    
    # 绘制训练曲线
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.axvline(x=best_epoch, color='r', linestyle='--', label=f'Best Model (Epoch {best_epoch+1})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'{model_name} Training History')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(save_dir, f'{model_name}_training_curve.png'))
    plt.close()
    
    # 恢复最佳模型状态
    model.load_state_dict(best_model_info['state_dict'])
    
    return model, best_model_info, history

# 辅助函数：将对象转换为JSON可序列化的格式
def convert_to_serializable(obj):
    """将对象转换为JSON可序列化的格式"""
    if isinstance(obj, torch.Tensor):
        # 将张量转换为Python原生类型
        obj = obj.detach().cpu().numpy()
        if obj.size == 1:
            return float(obj.item())  # 单个值转为float
        return obj.tolist()  # 数组转为列表
    elif isinstance(obj, np.ndarray):
        # 将NumPy数组转换为列表
        return obj.tolist()
    elif isinstance(obj, (np.float32, np.float64, np.int32, np.int64)):
        # 将NumPy标量转换为Python标量
        return float(obj) if np.issubdtype(obj.dtype, np.floating) else int(obj)
    elif isinstance(obj, dict):
        # 递归处理字典
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        # 递归处理列表
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, (float, int, str, bool, type(None))):
        # 这些类型已经是JSON可序列化的
        return obj
    else:
        # 其他类型，尝试转换为字符串
        try:
            return str(obj)
        except:
            return "Non-serializable object"

# ========================= 主训练代码 =========================

# 创建保存目录
os.makedirs('models', exist_ok=True)

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 模型参数设置
batch_size = 32
sequence_length = 24
forecast_horizon = 24
hidden_dim = 64
num_layers = 2
dropout = 0.2
learning_rate = 0.001
weight_decay = 1e-4
epochs = 10  # 源域预训练轮次

# 加载数据
print("加载所有类别的训练数据...")
train_loader, test_loader, categories = create_all_dataloaders(
    batch_size=batch_size, 
    sequence_length=sequence_length, 
    forecast_horizon=forecast_horizon
)

from torch.utils.data import Subset, DataLoader, random_split

# 从train_loader中获取完整训练建筑数据集
all_train_dataset = train_loader.dataset  # 这是ConcatDataset

# 设定划分比例（如80%训练，20%验证）
train_ratio = 0.8
val_ratio = 0.2
total_len = len(all_train_dataset)
train_len = int(total_len * train_ratio)
val_len = total_len - train_len

# 使用random_split划分
train_subset, val_subset = random_split(all_train_dataset, [train_len, val_len])

# 构建新的dataloader
batch_size = train_loader.batch_size
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# 获取数据维度信息
for inputs, targets, category, category_onehot in train_loader:
    print(f"输入形状: {inputs.shape}, 目标形状: {targets.shape}, 类别形状: {category_onehot.shape}")
    input_dim = inputs.shape[-1]
    num_buildings = inputs.shape[1]
    category_dim = category_onehot.shape[-1]
    break

# 创建DANN模型
print("创建和训练DANN模型...")

# 创建DANN模型
source_model = DANN(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    category_dim=category_dim,
    forecast_horizon=forecast_horizon,
    num_buildings=num_buildings,
    num_layers=num_layers,
    dropout=dropout,
    alpha=0.0  # 源域训练时不使用域对抗，设置alpha为0
)

# 训练源域模型
trained_source_model, source_best_info, source_history = train_dann_source_model(
    model=source_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=epochs,
    lr=learning_rate,
    weight_decay=weight_decay,
    model_name='dann_source_huber',
    save_dir='models',
    device=device
)

# 使用改进的完整评估函数对最终模型进行评估
print("\n评估源域模型性能...")
source_metrics = evaluate_model(
    model=trained_source_model, 
    test_loader=val_loader,
    model_name="DANN_Source_Huber_Model",
    device=device
)

# 提取重要的评估指标
print("\n源域模型评估指标:")
print(f"MAPE(%): {source_metrics['MAPE']:.2f}")
print(f"RMSD: {source_metrics['RMSD']:.4f}")
print(f"R²: {source_metrics['R2']:.4f}")
if 'CC' in source_metrics:
    print(f"CC: {source_metrics['CC']:.4f}")

# 打印源域的不确定性评估结果
if 'PICP' in source_metrics:
    print("\n源域不确定性评估:")
    print(f"预测区间覆盖率(PICP): {source_metrics['PICP']:.2f}% (目标95%)")
    print(f"校准误差: {source_metrics['calibration_error']:.2f}%")
    print(f"平均区间宽度(NMPIW): {source_metrics['NMPIW']:.4f}")
    print(f"不确定性质量分数(UQS): {source_metrics['UQS']:.4f}")

# 保存最终评估结果
metrics_path = os.path.join('models', 'dann_model_evaluation_metrics.json')
with open(metrics_path, 'w') as f:
    all_metrics = {
        'source_domain': {k: convert_to_serializable(v) 
                         for k, v in source_metrics.items() if k != 'Model'},
        'model_info': convert_to_serializable({
            'best_epoch': source_best_info['epoch'],
            'model_type': 'DANN'
        })
    }
    
    json.dump(all_metrics, f, indent=4)

print("训练完成！")

使用设备: cuda
加载所有类别的训练数据...
加载天气数据: Gator_mild_train.csv, 形状: (2184, 5)
加载天气数据: Gator_mild_test.csv, 形状: (6600, 5)
加载天气数据: Rat_mild_train.csv, 形状: (2184, 5)
加载天气数据: Rat_mild_test.csv, 形状: (6600, 5)
加载天气数据: Wolf_mild_train.csv, 形状: (2184, 5)
加载天气数据: Wolf_mild_test.csv, 形状: (6600, 5)
加载天气数据: Eagle_mild_train.csv, 形状: (2184, 5)
加载天气数据: Eagle_mild_test.csv, 形状: (6600, 5)
加载天气数据: Robin_mild_train.csv, 形状: (2184, 5)
加载天气数据: Robin_mild_test.csv, 形状: (6600, 5)
加载天气数据: Hog_mild_train.csv, 形状: (2184, 5)
加载天气数据: Hog_mild_test.csv, 形状: (6600, 5)
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 DO 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 DO 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 HO 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 HO 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 LI 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 LI 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 OF 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 OF 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 UL

Epoch 1/10 [Training]:   0%|          | 0/267 [00:00<?, ?it/s]

Epoch 1/10 [Validation]:   0%|          | 0/67 [00:00<?, ?it/s]

✅ 保存最佳模型 (epoch 1), 验证损失: 0.0085
Epoch 1/10 - Train Loss: 0.0190, Val Loss: 0.0085, RMSD: 0.1305, R²: 0.5359, Best Val Loss: 0.0085 (Epoch 1), LR: 0.001000


Epoch 2/10 [Training]:   0%|          | 0/267 [00:00<?, ?it/s]

...

Epoch 10/10 [Training]:   0%|          | 0/267 [00:00<?, ?it/s]

Epoch 10/10 [Validation]:   0%|          | 0/67 [00:00<?, ?it/s]

Epoch 10/10 - Train Loss: 0.0047, Val Loss: 0.0040, RMSD: 0.0897, R²: 0.7809, Best Val Loss: 0.0039 (Epoch 9), LR: 0.001000

评估源域模型性能...


2025-06-23 23:05:44,488 - WARNING - 特征数量过大，进行采样: source_features.shape=torch.Size([10680, 64]), target_features.shape=torch.Size([10680, 64])
2025-06-23 23:05:44,492 - INFO - 采样后: source_features.shape=torch.Size([5000, 64]), target_features.shape=torch.Size([5000, 64])


形状信息:
预测: (256320,)
目标: (256320,)
下界: (256320,)
上界: (256320,)
不确定性: (256320,)

DANN_Source_Huber_Model 评估报告:
RMSD: 0.08909539903381088
MAPE: 16.58074188232422%
R²: 0.7837463617324829
KL散度: N/A

迁移学习评估:
域间距离 (A-distance): 0.09280002117156982
特征对齐质量: 0.30181360244750977
MMD: 0.00023987889289855957

不确定性评估:
预测区间覆盖率(PICP): 69.80% (目标95%)
平均预测区间宽度(NMPIW): 0.14853931963443756
校准误差: 25.20%
平均不确定性(标准差): 0.03841352090239525

源域模型评估指标:
MAPE(%): 16.58
RMSD: 0.0891
R²: 0.7837
CC: 0.8858

源域不确定性评估:
预测区间覆盖率(PICP): 69.80% (目标95%)
校准误差: 25.20%
平均区间宽度(NMPIW): 0.1485
不确定性质量分数(UQS): 0.5009
训练完成！


In [ ]:
import os
import json
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import bisect
from bilstm import BaselineBiLSTM

def create_combined_dataloaders(source_category, target_category, data_shortage, batch_size, sequence_length, forecast_horizon):
    """
    构建合并数据加载器（源域：mild，目标域：指定缺失程度）
    对于CC类别，使用所有其他类别的训练数据作为源域
    """
    print(f"为类别 {target_category} 创建数据加载器 (shortage: {data_shortage})...")
    
    try:
        # 正常类别的处理逻辑（非CC类别）
        if train_test_labels[target_category]["train"] is not None:
            # 源域使用完整 mild 数据
            source_train_loader, source_test_loader, _ = create_category_dataloaders(
                category=target_category,
                batch_size=batch_size,
                sequence_length=sequence_length,
                forecast_horizon=forecast_horizon,
                data_shortage='mild'
            )

            # 目标域使用指定数据缺失场景
            target_train_loader, target_test_loader, _ = create_category_dataloaders(
                category=target_category,
                batch_size=batch_size,
                sequence_length=sequence_length,
                forecast_horizon=forecast_horizon,
                data_shortage=data_shortage
            )

            # 检查数据加载器是否为None
            if source_train_loader is None or target_train_loader is None:
                print(f"警告：无法创建 {target_category} 的数据加载器")
                return None, None, None, categories

            class SingleBuildingWrapper(Dataset):
                """确保所有样本都只包含一个建筑的数据集包装器"""
                def __init__(self, dataset):
                    self.dataset = dataset

                def __len__(self):
                    return len(self.dataset)

                def __getitem__(self, idx):
                    # 获取原始样本
                    data = self.dataset[idx]
                    if isinstance(data, tuple) and len(data) >= 2:
                        inputs, targets = data[0], data[1]
                        
                        # 检查并确保只使用一个建筑
                        if inputs.shape[0] > 1:
                            inputs = inputs[0:1]  # 只保留第一个建筑
                            targets = targets[0:1]  # 同样只保留第一个建筑的目标
                        
                        # 重建tuple
                        result = (inputs,) + (targets,) + data[2:]
                        return result
                    else:
                        return data

            # 使用包装器处理每个数据集
            source_train_dataset = SingleBuildingWrapper(source_train_loader.dataset)
            source_test_dataset = SingleBuildingWrapper(source_test_loader.dataset)
            target_train_dataset = SingleBuildingWrapper(target_train_loader.dataset)
            target_test_dataset = SingleBuildingWrapper(target_test_loader.dataset)

            class CombinedDataset(Dataset):
                def __init__(self, datasets):
                    self.datasets = datasets
                    self.lengths = [len(ds) for ds in datasets]
                    self.cumulative_lengths = [0]
                    for length in self.lengths:
                        self.cumulative_lengths.append(self.cumulative_lengths[-1] + length)

                def __len__(self):
                    return sum(self.lengths)

                def __getitem__(self, idx):
                    dataset_idx = bisect.bisect_right(self.cumulative_lengths, idx) - 1
                    sample_idx = idx - self.cumulative_lengths[dataset_idx]
                    return self.datasets[dataset_idx][sample_idx]

            # 合并处理后的数据集
            combined_dataset = CombinedDataset([
                source_train_dataset,
                source_test_dataset,
                target_train_dataset
            ])

            # 创建新的数据加载器
            combined_train_loader = DataLoader(
                combined_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0
            )
            
            # 目标测试集也需要使用单建筑格式
            target_test_loader_single = DataLoader(
                target_test_dataset,
                batch_size=batch_size,
                shuffle=False,
                num_workers=0
            )
            
            # 目标训练集也需要使用单建筑格式
            target_train_loader_single = DataLoader(
                target_train_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0
            )

            print(f"成功创建 {target_category} 的数据加载器，共 {len(combined_dataset)} 个样本")
            return combined_train_loader, target_test_loader_single, target_train_loader_single, categories
        
        # CC类别的特殊处理
        else:
            print(f"类别 {target_category} 没有训练数据，使用特殊处理")
            
            # 收集所有其他类别的训练数据作为源域
            all_source_datasets = []
            
            for cat in categories:
                if cat != target_category and train_test_labels[cat]["train"] is not None:
                    # 加载其他类别的训练数据（使用mild数据）
                    cat_train_loader, _, _ = create_category_dataloaders(
                        category=cat,
                        batch_size=batch_size,
                        sequence_length=sequence_length,
                        forecast_horizon=forecast_horizon,
                        data_shortage='mild'  # 源域使用完整数据
                    )
                    if cat_train_loader is not None:
                        all_source_datasets.append(cat_train_loader.dataset)
            
            # 获取CC类别的测试数据（作为目标域）
            _, cc_test_loader, _ = create_category_dataloaders(
                category=target_category,
                batch_size=batch_size,
                sequence_length=sequence_length,
                forecast_horizon=forecast_horizon,
                data_shortage=data_shortage
            )
            
            if not all_source_datasets or cc_test_loader is None:
                print(f"警告：无法为CC类别创建数据加载器")
                return None, None, None, categories
            
            # 获取CC测试建筑ID
            cc_test_building = train_test_labels[target_category]["test"][0]
            
            # 创建一个简单的单建筑数据集作为训练集
            class EmptyDataset(Dataset):
                def __len__(self):
                    return 10  # 小数据集
                
                def __getitem__(self, idx):
                    # 创建一个兼容的数据项
                    x = torch.zeros((1, sequence_length, input_dim))  # [1建筑, 序列长度, 特征维度]
                    y = torch.zeros((1, forecast_horizon))  # [1建筑, 预测长度]
                    category = target_category
                    category_onehot = torch.zeros(len(categories))
                    category_idx = categories.index(target_category)
                    category_onehot[category_idx] = 1.0
                    return x, y, category, category_onehot
            
            # 使用包装器确保数据格式一致
            class SingleBuildingWrapper(Dataset):
                def __init__(self, dataset):
                    self.dataset = dataset

                def __len__(self):
                    return len(self.dataset)

                def __getitem__(self, idx):
                    data = self.dataset[idx]
                    if isinstance(data, tuple) and len(data) >= 2:
                        inputs, targets = data[0], data[1]
                        if inputs.shape[0] > 1:
                            inputs = inputs[0:1]
                            targets = targets[0:1]
                        result = (inputs,) + (targets,) + data[2:]
                        return result
                    else:
                        return data
            
            # 包装所有源域数据集
            wrapped_source_datasets = [SingleBuildingWrapper(ds) for ds in all_source_datasets]
            
            # 创建空的训练集作为CC的训练数据
            cc_train_dataset = EmptyDataset()
            wrapped_cc_train = SingleBuildingWrapper(cc_train_dataset)
            wrapped_cc_test = SingleBuildingWrapper(cc_test_loader.dataset)
            
            # 合并所有源域数据和CC的训练数据
            class CombinedDataset(Dataset):
                def __init__(self, datasets):
                    self.datasets = datasets
                    self.lengths = [len(ds) for ds in datasets]
                    self.cumulative_lengths = [0]
                    for length in self.lengths:
                        self.cumulative_lengths.append(self.cumulative_lengths[-1] + length)

                def __len__(self):
                    return sum(self.lengths)

                def __getitem__(self, idx):
                    dataset_idx = bisect.bisect_right(self.cumulative_lengths, idx) - 1
                    sample_idx = idx - self.cumulative_lengths[dataset_idx]
                    return self.datasets[dataset_idx][sample_idx]
            
            # combined_train_loader = 所有其他类别的训练数据 + CC的训练时间段数据
            combined_dataset = CombinedDataset(wrapped_source_datasets + [wrapped_cc_train])
            combined_train_loader = DataLoader(
                combined_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0
            )
            
            # target_test_loader_single = CC的测试数据
            target_test_loader_single = DataLoader(
                wrapped_cc_test,
                batch_size=batch_size,
                shuffle=False,
                num_workers=0
            )
            
            # target_train_loader_single = CC的训练时间段数据
            target_train_loader_single = DataLoader(
                wrapped_cc_train,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0
            )
            
            print(f"CC类别数据加载器创建成功:")
            print(f"  - combined_train_loader: {len(combined_dataset)} 样本 (包含所有其他类别 + CC训练时间段)")
            print(f"  - target_test_loader: {len(wrapped_cc_test)} 样本")
            print(f"  - target_train_loader: {len(wrapped_cc_train)} 样本")
            
            return combined_train_loader, target_test_loader_single, target_train_loader_single, categories
    
    except Exception as e:
        print(f"创建数据加载器时出错: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None, categories

def adapt_to_target_domain_dann(source_model, source_loader, target_loader, epochs=20, lr=0.001, 
                              device='cuda', lambda_domain=0.4, early_stopping_patience=3):
    """
    使用DANN方法将源域模型适应到目标域
    
    参数:
    - source_model: 预训练的源域模型
    - source_loader: 源域数据加载器
    - target_loader: 目标域数据加载器
    - epochs: 迁移训练轮数
    - lr: 学习率
    - device: 计算设备
    - lambda_domain: 域对抗损失权重
    - early_stopping_patience: 早停耐心值
    
    返回:
    - model: 适应后的模型
    - history: 训练历史
    """
    # 配置日志
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('dann_transfer_learning.log'),
            logging.StreamHandler()
        ]
    )
    
    # 忽略警告
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    
    # 深拷贝源模型
    model = copy.deepcopy(source_model).to(device)
    
    # 创建域分类器
    domain_classifier = DomainClassifier(
        feature_dim=model.hidden_dim,
        hidden_dim=64,
        dropout=0.3
    ).to(device)
    
    # 设置优化器
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    optimizer_disc = optim.AdamW(domain_classifier.parameters(), lr=lr*0.5, weight_decay=0.02)
    
    # 设置学习率调度器
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=5, T_mult=2, eta_min=lr*0.1
    )
    
    # 设置损失函数
    huber_loss_fn = HuberLoss(delta=1.0)
    domain_criterion = nn.BCEWithLogitsLoss()
    
    # 初始化训练记录
    best_rmse = float('inf')
    patience_counter = 0
    best_model_state = None
    history = {'total_loss': [], 'task_loss': [], 'domain_loss': [], 'rmse': []}
    
    # 梯度裁剪阈值
    max_grad_norm = 1.0
    
    logging.info("开始DANN域自适应迁移学习训练...")
    
    # 主训练循环
    for epoch in range(epochs):
        model.train()
        domain_classifier.train()
        
        # 动态调整域对抗强度
        p = float(epoch) / epochs
        alpha = 2. / (1. + np.exp(-10 * p)) - 1  # 从0逐渐增加到1
        model.set_alpha(alpha * lambda_domain)
        
        # 初始化统计变量
        epoch_stats = {'total_loss': 0, 'task_loss': 0, 'domain_loss': 0}
        
        # 批次训练循环
        n_batches = min(len(source_loader), len(target_loader))
        progress_bar = tqdm(range(n_batches), desc=f'Epoch {epoch+1}/{epochs}')
        
        for batch_idx in progress_bar:
            try:
                # 获取源域和目标域的一个批次数据
                source_batch = next(iter(source_loader))
                target_batch = next(iter(target_loader))
                
                # 解析数据并移动到计算设备
                source_inputs, source_targets, _, source_category = [
                    x.float().to(device) if torch.is_tensor(x) else x for x in source_batch
                ]
                target_inputs, target_targets, _, target_category = [
                    x.float().to(device) if torch.is_tensor(x) else x for x in target_batch
                ]
                
                # 1. 特征提取
                source_features = model.get_features(source_inputs)
                target_features = model.get_features(target_inputs)
                
                # 2. 域对抗训练
                # 准备域标签（源域=0，目标域=1）
                batch_size = source_features.size(0)
                source_domain_labels = torch.zeros(batch_size, 1).to(device)
                target_domain_labels = torch.ones(batch_size, 1).to(device)
                
                # 连接特征和标签
                features = torch.cat([source_features, target_features], dim=0)
                domain_labels = torch.cat([source_domain_labels, target_domain_labels], dim=0)
                
                # 域分类器预测
                domain_preds = domain_classifier(model.grl(features))
                domain_loss = domain_criterion(domain_preds, domain_labels)
                
                # 3. 任务预测
                source_predictions = model(source_inputs, source_category)
                target_predictions = model(target_inputs, target_category)
                
                # 4. 计算任务损失
                source_task_loss = huber_loss_fn(source_predictions, source_targets)
                target_task_loss = huber_loss_fn(target_predictions, target_targets)
                
                # 5. 动态平衡源域和目标域任务权重
                src_weight = max(0.2, 1.0 - epoch/epochs)    # 源域权重从1线性衰减到0.2
                tgt_weight = min(5.0, 1.0 + epoch*4/epochs)   # 目标域权重从1线性增加到5
                
                # 6. 计算总损失
                task_loss = source_task_loss * src_weight + target_task_loss * tgt_weight
                total_loss = task_loss + domain_loss * model.alpha
                
                # 7. 反向传播与优化
                optimizer.zero_grad()
                optimizer_disc.zero_grad()
                total_loss.backward()
                
                # 梯度裁剪
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                torch.nn.utils.clip_grad_norm_(domain_classifier.parameters(), max_grad_norm)
                
                # 更新参数
                optimizer.step()
                optimizer_disc.step()
                
                # 统计损失
                epoch_stats['total_loss'] += total_loss.item()
                epoch_stats['task_loss'] += task_loss.item()
                epoch_stats['domain_loss'] += domain_loss.item()
                
                # 更新进度条
                progress_bar.set_postfix({
                    'loss': f"{total_loss.item():.4f}",
                    'task': f"{task_loss.item():.4f}",
                    'domain': f"{domain_loss.item():.4f}",
                    'alpha': f"{model.alpha:.4f}"
                })
                
            except Exception as e:
                logging.warning(f"批次处理出错: {str(e)}")
                continue
        
        # 更新学习率
        scheduler.step()
        
        # 计算平均损失
        avg_total_loss = epoch_stats['total_loss'] / n_batches
        avg_task_loss = epoch_stats['task_loss'] / n_batches
        avg_domain_loss = epoch_stats['domain_loss'] / n_batches
        
        # 记录训练历史
        history['total_loss'].append(avg_total_loss)
        history['task_loss'].append(avg_task_loss)
        history['domain_loss'].append(avg_domain_loss)
        
        # 评估模型
        current_rmse = evaluate_dann_on_target(model, target_loader, device)
        history['rmse'].append(current_rmse)
        
        logging.info(
            f"Epoch {epoch+1}/{epochs} - "
            f"Loss: {avg_total_loss:.4f}, "
            f"Task: {avg_task_loss:.4f}, "
            f"Domain: {avg_domain_loss:.4f}, "
            f"RMSE: {current_rmse:.4f}, "
            f"Alpha: {model.alpha:.4f}"
        )
        
        # 早停检查
        if current_rmse < best_rmse:
            best_rmse = current_rmse
            best_model_state = copy.deepcopy(model.state_dict())
            patience_counter = 0
            logging.info(f"发现新的最佳RMSE: {best_rmse:.4f}")
        else:
            patience_counter += 1
            logging.info(f"未改进RMSE，耐心值: {patience_counter}/{early_stopping_patience}")
            
            if patience_counter >= early_stopping_patience:
                logging.info(f"触发早停，在epoch {epoch+1}")
                break
    
    # 恢复最佳模型
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        logging.info(f"已恢复最佳模型")
    
    # 保存模型
    try:
        checkpoint_path = f'models/dann_adapted_model.pth'
        torch.save({
            'state_dict': model.state_dict(),
            'domain_classifier': domain_classifier.state_dict(),
            'history': history,
            'best_rmse': best_rmse
        }, checkpoint_path)
        logging.info(f"模型已保存到 {checkpoint_path}")
    except Exception as e:
        logging.error(f"保存模型失败: {str(e)}")
    
    return model, history

def evaluate_dann_on_target(model, target_loader, device):
    """
    在目标域数据上评估DANN模型的RMSE
    """
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets, _, category in target_loader:
            inputs = inputs.float().to(device)
            targets = targets.float().to(device)
            category = category.float().to(device)
            
            # 前向传播
            predictions = model(inputs, category)
            
            # 收集预测和目标
            all_preds.append(predictions.cpu())
            all_targets.append(targets.cpu())
    
    # 合并所有批次
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    
    # 计算RMSE
    mse = torch.mean((all_preds - all_targets) ** 2)
    rmse = torch.sqrt(mse).item()
    
    return rmse

# ========================= 主程序 =========================

# 模型参数设置
batch_size = 32
sequence_length = 24
forecast_horizon = 24
hidden_dim = 64
num_layers = 2
dropout = 0.3
learning_rate = 0.001
weight_decay = 1e-5
transfer_epochs = 15
input_dim = 6

# 定义域索引映射
DOMAIN_MAPPING = {
    "DO": 1,
    "HO": 2,
    "LI": 3,
    "OF": 4,
    "UL": 5,
    "CC": None
}

# 数据缺失场景
data_shortage_scenarios = ['mild', 'heavy', 'extreme']
with open('train_test_labels.json', 'r') as f:
    train_test_labels = json.load(f)
# 加载数据
print("加载所有类别的训练数据...")
train_loader, test_loader, categories = create_all_dataloaders(
    batch_size=batch_size, 
    sequence_length=sequence_length, 
    forecast_horizon=forecast_horizon
)

# 获取数据维度信息
for inputs, targets, category, category_onehot in train_loader:
    print(f"输入形状: {inputs.shape}, 目标形状: {targets.shape}, 类别形状: {category_onehot.shape}")
    input_dim = inputs.shape[-1]
    num_buildings = inputs.shape[1]
    category_dim = category_onehot.shape[-1]
    break

# 创建DANN模型
print("创建DANN模型...")
source_model = DANN(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    category_dim=category_dim,
    forecast_horizon=forecast_horizon,
    num_buildings=num_buildings,
    num_layers=num_layers,
    dropout=dropout,
    alpha=0.0  # 源域训练时不使用域对抗
)

# 训练源域模型
print("训练源域DANN模型...")
# 这里可以使用您原有的训练函数，只需将模型替换为DANN模型

# 迁移学习循环
transfer_results = {}
source_domain_idx = 0

for target_category in categories:
    transfer_results[target_category] = {}
    target_domain_idx = DOMAIN_MAPPING[target_category]
    
    for data_shortage in data_shortage_scenarios:
        print(f"\n🚀 [Transfer Learning] Source(ALL) ➜ Target({target_category}), Shortage: {data_shortage}")
        
        try:
            # 初始化结果字典
            transfer_results[target_category][data_shortage] = {}
            
            # 加载数据
            combined_train_loader, target_test_loader, target_train_loader, _ = create_combined_dataloaders(
                source_category=target_category,
                target_category=target_category,
                data_shortage=data_shortage,
                batch_size=batch_size,
                sequence_length=sequence_length,
                forecast_horizon=forecast_horizon
            )
            
            # 检查数据加载器
            if combined_train_loader is None or target_test_loader is None:
                print(f"⚠️ 无法为 {target_category}/{data_shortage} 创建数据加载器，跳过")
                continue

            # 使用DANN进行迁移学习
            print(f"🔄 使用DANN进行迁移学习 for category {target_category}...")
            adapted_model, transfer_history = adapt_to_target_domain_dann(
                source_model=source_model,
                source_loader=combined_train_loader,
                target_loader=target_test_loader,
                epochs=transfer_epochs,
                lr=learning_rate / 2,
                device=device,
                lambda_domain=0.4,
                early_stopping_patience=3
            )
            
            # 评估DANN模型（适应evaluate_model的修改）
            print(f"📊 评估DANN模型 for category {target_category}...")
            dann_metrics = evaluate_model(
                model=adapted_model,
                test_loader=target_test_loader,
                model_name=f"DANN_TL_ALL_to_{target_category}_{data_shortage}",
                baseline_model=None,  # 保持基线模型参数
                device=device
            )
            
            # 保存DANN模型结果（更新保存逻辑）
            transfer_results[target_category][data_shortage]['dann'] = {
                'metrics': dann_metrics,
                'model_type': "dann",
                # 新增：提取并保存特定指标
                'rmsd': dann_metrics.get('RMSD', 'N/A'),
                'cv_rmse': dann_metrics.get('CV-RMSE', 'N/A'),
                'sd_real': dann_metrics.get('SD_real', 'N/A'),
                'sd_pred': dann_metrics.get('SD_pred', 'N/A'),
                'cc': dann_metrics.get('CC', 'N/A'),
                # 迁移学习增益指标
                'transfer_gain': dann_metrics.get('transfer_gain', 'N/A'),
                'is_negative_transfer': dann_metrics.get('is_negative_transfer', 'N/A')
            }
            
            # 打印指标对比表（更新表格生成逻辑）
            print("\n📊 基线模型 vs DANN模型 指标对比:")
            print_metrics_table([
                {
                    'name': 'Baseline',
                    'metrics': calculate_metrics
                },
                {
                    'name': 'DANN',
                    'metrics': dann_metrics
                }
            ])

        except Exception as e:
            print(f"❌ Error: {target_category}/{data_shortage} transfer failed: {e}")
            import traceback
            traceback.print_exc()
            continue

print("\n✅ 所有迁移学习实验完成")

加载所有类别的训练数据...
加载天气数据: Gator_mild_train.csv, 形状: (2184, 5)
加载天气数据: Gator_mild_test.csv, 形状: (6600, 5)
加载天气数据: Rat_mild_train.csv, 形状: (2184, 5)
加载天气数据: Rat_mild_test.csv, 形状: (6600, 5)
加载天气数据: Wolf_mild_train.csv, 形状: (2184, 5)
加载天气数据: Wolf_mild_test.csv, 形状: (6600, 5)
加载天气数据: Eagle_mild_train.csv, 形状: (2184, 5)
加载天气数据: Eagle_mild_test.csv, 形状: (6600, 5)
加载天气数据: Robin_mild_train.csv, 形状: (2184, 5)
加载天气数据: Robin_mild_test.csv, 形状: (6600, 5)
加载天气数据: Hog_mild_train.csv, 形状: (2184, 5)
加载天气数据: Hog_mild_test.csv, 形状: (6600, 5)
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 DO 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 DO 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 HO 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 HO 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 LI 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 LI 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 OF 训练数据集长度: 2136
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
类别 OF 测试数据集长度: 6552
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
类别 UL 训练数据集长度: 2

2025-06-23 23:05:45,619 - INFO - 开始DANN域自适应迁移学习训练...


输入形状: torch.Size([32, 5, 24, 6]), 目标形状: torch.Size([32, 5, 24]), 类别形状: torch.Size([32, 6])
创建DANN模型...
训练源域DANN模型...

🚀 [Transfer Learning] Source(ALL) ➜ Target(DO), Shortage: mild
为类别 DO 创建数据加载器 (shortage: mild)...
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
Chronos数据集初始化: 5 个建筑, 2136 个有效样本
Chronos数据集初始化: 1 个建筑, 6552 个有效样本
成功创建 DO 的数据加载器，共 10824 个样本
🔄 使用DANN进行迁移学习 for category DO...


Epoch 1/15:   0%|          | 0/205 [00:00<?, ?it/s]

2025-06-23 23:06:30,757 - INFO - Epoch 1/15 - Loss: 0.0431, Task: 0.0431, Domain: 0.6934, RMSE: 0.1251, Alpha: 0.0000
2025-06-23 23:06:30,760 - INFO - 发现新的最佳RMSE: 0.1251


...

Epoch 2/15:   0%|          | 0/205 [00:00<?, ?it/s]

2025-06-24 00:20:15,658 - INFO - Epoch 2/15 - Loss: 0.1096, Task: 0.0205, Domain: 0.6929, RMSE: 0.2021, Alpha: 0.1286
2025-06-24 00:20:15,659 - INFO - 未改进RMSE，耐心值: 1/3


Epoch 3/15:   0%|          | 0/205 [00:00<?, ?it/s]

2025-06-24 00:21:23,645 - INFO - Epoch 3/15 - Loss: 0.1785, Task: 0.0169, Domain: 0.6930, RMSE: 0.2116, Alpha: 0.2331
2025-06-24 00:21:23,648 - INFO - 未改进RMSE，耐心值: 2/3


Epoch 4/15:   0%|          | 0/205 [00:00<?, ?it/s]

In [ ]:
# ========================= 保存结果 =========================

# 保存迁移学习结果到JSON文件
try:
    # 将结果转换为可序列化格式
    def convert_to_serializable(obj):
        """将对象转换为JSON可序列化的格式"""
        if isinstance(obj, torch.Tensor):
            # 将张量转换为Python原生类型
            obj = obj.detach().cpu().numpy()
            if obj.size == 1:
                return float(obj.item())  # 单个值转为float
            return obj.tolist()  # 数组转为列表
        elif isinstance(obj, np.ndarray):
            # 将NumPy数组转换为列表
            return obj.tolist()
        elif isinstance(obj, (np.float32, np.float64, np.int32, np.int64)):
            # 将NumPy标量转换为Python标量
            return float(obj) if np.issubdtype(obj.dtype, np.floating) else int(obj)
        elif isinstance(obj, dict):
            # 递归处理字典
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            # 递归处理列表
            return [convert_to_serializable(item) for item in obj]
        elif isinstance(obj, (float, int, str, bool, type(None))):
            # 这些类型已经是JSON可序列化的
            return obj
        else:
            # 其他类型，尝试转换为字符串
            try:
                return str(obj)
            except:
                return "Non-serializable object"
    
    # 转换结果为可序列化格式
    serializable_results = convert_to_serializable(transfer_results)
    
    # 保存到JSON文件
    results_path = 'results/dann_transfer_results.json'
    os.makedirs(os.path.dirname(results_path), exist_ok=True)
    with open(results_path, 'w') as f:
        json.dump(serializable_results, f, indent=4)
    
    print(f"✅ 迁移学习结果已保存到 {results_path}")
    
    # 保存结果到CSV文件
    # 准备CSV数据（新增SD、CV-RMSE等指标）
    csv_data = []
    csv_headers = [
        'Category', 'Data_Shortage', 'Model_Type', 
        'RMSD', 'MAPE', 'R2', 'CV-RMSE', 'SD_real', 'SD_pred', 'CC', 
        'PIR', 'PICP', 'NMPIW', 'Calibration_Error', 'UQS', 'Avg_Uncertainty',
        'A_distance', 'Feature_Alignment', 'MMD', 
        'Is_Negative_Transfer', 'Transfer_Gain', 'Sample_Efficiency'
    ]
    
    for category, category_results in transfer_results.items():
        for shortage, shortage_results in category_results.items():
            for model_type, model_data in shortage_results.items():
                if 'metrics' in model_data:
                    metrics = model_data['metrics']
                    row = {
                        'Category': category,
                        'Data_Shortage': shortage,
                        'Model_Type': model_type,
                        'RMSD': metrics.get('RMSD', 'N/A'),
                        'MAPE': metrics.get('MAPE', 'N/A'),
                        'R2': metrics.get('R2', 'N/A'),
                        'CV-RMSE': metrics.get('CV-RMSE', 'N/A'),
                        'SD_real': metrics.get('SD_real', 'N/A'),
                        'SD_pred': metrics.get('SD_pred', 'N/A'),
                        'CC': metrics.get('CC', 'N/A'),
                        'PIR': metrics.get('PIR', 'N/A'),
                        'PICP': metrics.get('PICP', 'N/A'),
                        'NMPIW': metrics.get('NMPIW', 'N/A'),
                        'Calibration_Error': metrics.get('calibration_error', 'N/A'),
                        'UQS': metrics.get('UQS', 'N/A'),
                        'Avg_Uncertainty': metrics.get('avg_uncertainty', 'N/A'),
                        'A_distance': metrics.get('a_distance', 'N/A'),
                        'Feature_Alignment': metrics.get('feature_alignment', 'N/A'),
                        'MMD': metrics.get('mmd', 'N/A'),
                        'Is_Negative_Transfer': metrics.get('is_negative_transfer', 'N/A'),
                        'Transfer_Gain': metrics.get('transfer_gain', 'N/A'),
                        'Sample_Efficiency': metrics.get('sample_efficiency', 'N/A')
                    }
                    csv_data.append(row)
    
    # 创建DataFrame并保存为CSV
    csv_path = 'results/dann_transfer_results.csv'
    df = pd.DataFrame(csv_data)
    df.to_csv(csv_path, index=False)
    print(f"✅ 迁移学习结果已保存到CSV文件: {csv_path}")
    
    # 为每个类别创建单独的CSV文件
    for category in transfer_results.keys():
        category_data = [row for row in csv_data if row['Category'] == category]
        if category_data:
            category_csv_path = f'results/dann_transfer_results_{category}.csv'
            pd.DataFrame(category_data).to_csv(category_csv_path, index=False)
            print(f"✅ 类别 {category} 的结果已保存到: {category_csv_path}")
    
    # 创建性能对比汇总CSV（新增SD、CV-RMSE对比）
    summary_data = []
    for shortage in data_shortage_scenarios:
        for category in transfer_results.keys():
            if shortage in transfer_results[category]:
                baseline_metrics = {}
                dann_metrics = {}
                
                if ('baseline' in transfer_results[category][shortage] and 
                    'metrics' in transfer_results[category][shortage]['baseline']):
                    baseline_metrics = transfer_results[category][shortage]['baseline']['metrics']
                
                if ('dann' in transfer_results[category][shortage] and 
                    'metrics' in transfer_results[category][shortage]['dann']):
                    dann_metrics = transfer_results[category][shortage]['dann']['metrics']
                
                # 计算各指标改进百分比
                def calculate_improvement(baseline, dann):
                    if baseline != 'N/A' and dann != 'N/A' and baseline != 0:
                        return (baseline - dann) / baseline * 100
                    return 'N/A'
                
                # 提取关键指标
                baseline_rmsd = baseline_metrics.get('RMSD', 'N/A')
                dann_rmsd = dann_metrics.get('RMSD', 'N/A')
                rmsd_improvement = calculate_improvement(baseline_rmsd, dann_rmsd)
                
                baseline_cv_rmse = baseline_metrics.get('CV-RMSE', 'N/A')
                dann_cv_rmse = dann_metrics.get('CV-RMSE', 'N/A')
                cv_rmse_improvement = calculate_improvement(baseline_cv_rmse, dann_cv_rmse)
                
                baseline_sd = baseline_metrics.get('SD_pred', 'N/A')
                dann_sd = dann_metrics.get('SD_pred', 'N/A')
                sd_improvement = calculate_improvement(baseline_sd, dann_sd) if baseline_sd != 'N/A' and dann_sd != 'N/A' else 'N/A'
                
                baseline_cc = baseline_metrics.get('CC', 'N/A')
                dann_cc = dann_metrics.get('CC', 'N/A')
                cc_improvement = calculate_improvement(baseline_cc, dann_cc) if baseline_cc != 'N/A' and dann_cc != 'N/A' else 'N/A'
                
                summary_data.append({
                    'Category': category,
                    'Data_Shortage': shortage,
                    'Baseline_RMSD': baseline_rmsd,
                    'DANN_RMSD': dann_rmsd,
                    'RMSD_Improvement_Percent': rmsd_improvement,
                    'Baseline_CV-RMSE': baseline_cv_rmse,
                    'DANN_CV-RMSE': dann_cv_rmse,
                    'CV-RMSE_Improvement_Percent': cv_rmse_improvement,
                    'Baseline_SD': baseline_sd,
                    'DANN_SD': dann_sd,
                    'SD_Improvement_Percent': sd_improvement,
                    'Baseline_CC': baseline_cc,
                    'DANN_CC': dann_cc,
                    'CC_Improvement_Percent': cc_improvement
                })
    
    # 保存性能对比汇总
    summary_csv_path = 'results/dann_performance_summary.csv'
    pd.DataFrame(summary_data).to_csv(summary_csv_path, index=False)
    print(f"✅ 性能对比汇总已保存到: {summary_csv_path}")
    
except Exception as e:
    print(f"❌ 保存结果失败: {e}")
    import traceback
    traceback.print_exc()

print("\n🎉 DANN迁移学习实验全部完成！")